In [ ]:
!pip install flask flask_cors requests inference_sdk
from flask import Flask, request, jsonify
import cv2
import numpy as np
import requests
from inference_sdk import InferenceHTTPClient

app = Flask(__name__)

# Roboflow API Client (Replace with your API key)
CLIENT = InferenceHTTPClient(api_url="https://detect.roboflow.com", api_key="df5H1Hm13kb0WYd0Yhdc")

# ESP32 Addresses
ESP32_CAM_IP = "http://your-esp32cam-ip/"
ESP32_ROBOT_IP = "http://your-esp32robot-ip/"

@app.route('/upload', methods=['POST'])
def upload():
    try:
        # Read image
        image = np.frombuffer(request.data, np.uint8)
        img = cv2.imdecode(image, cv2.IMREAD_COLOR)

        # Send image for inference
        result = CLIENT.infer(img, model_id="garbage-classification-3/2")
        predictions = result.get('predictions', [])

        if predictions:
            best_pred = max(predictions, key=lambda p: p['confidence'])
            label = best_pred['class']
            confidence = best_pred['confidence']

            if label.lower() in ["trash", "garbage"]:
                send_signal(ESP32_CAM_IP, "TRASH_DETECTED")
                send_signal(ESP32_ROBOT_IP, "PICK_TRASH")
                return jsonify({"message": "Trash detected, robot activated!"})
            else:
                return jsonify({"message": "No trash detected."})

        return jsonify({"message": "No object detected."})

    except Exception as e:
        return jsonify({"error": str(e)})

def send_signal(ip, message):
    """ Sends signal to ESP32 """
    try:
        requests.post(ip, json={"signal": message})
        print(f"Sent signal {message} to {ip}")
    except Exception as e:
        print(f"Error sending signal: {e}")

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.13
    Uninstalling aiohttp-3.11.13:
      Successfully uninstalled aiohttp-3.11.13


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
